# Tier 3: Cross-Library Validation

**Learning Objectives:**
- Validate annuity-pricing against external libraries
- Understand acceptable tolerance levels
- Build confidence in pricing accuracy

**Validation Libraries:**
- financepy: Options pricing
- QuantLib: Yield curves
- pyfeng: SABR, Heston

**Duration:** ~20 minutes

**Note:** This notebook requires optional validation dependencies:
```bash
pip install annuity-pricing[validation]
```

In [ ]:
import numpy as np
from annuity_pricing.options.pricing.black_scholes import (
    black_scholes_call,
    black_scholes_put,
)

## 1. Hull Textbook Validation [T1]

Hull Example 15.6 provides known-answer test values.

In [ ]:
# Hull Example 15.6
S = 42.0
K = 40.0
r = 0.10
q = 0.0
sigma = 0.20
T = 0.5

# Expected values from Hull
expected_call = 4.76
expected_put = 0.81

# Our values
our_call = black_scholes_call(S, K, r, q, sigma, T)
our_put = black_scholes_put(S, K, r, q, sigma, T)

print("Hull Example 15.6 Validation:")
print(f"{'':20} {'Expected':>12} {'Ours':>12} {'Diff':>12}")
print("-" * 58)
print(f"{'Call Price':20} ${expected_call:>11.2f} ${our_call:>11.4f} ${abs(our_call - expected_call):>11.4f}")
print(f"{'Put Price':20} ${expected_put:>11.2f} ${our_put:>11.4f} ${abs(our_put - expected_put):>11.4f}")

# Validate
call_ok = abs(our_call - expected_call) < 0.01
put_ok = abs(our_put - expected_put) < 0.01
print(f"\nStatus: {'✓ PASS' if call_ok and put_ok else '✗ FAIL'}")

## 2. financepy Validation (Optional)

In [ ]:
try:
    from financepy.products.equity.equity_vanilla_option import EquityVanillaOption
    from financepy.utils.global_types import OptionTypes
    from financepy.utils.date import Date
    from financepy.market.curves.discount_curve_flat import DiscountCurveFlat
    from financepy.models.black_scholes import BlackScholes
    
    FINANCEPY_AVAILABLE = True
    print("financepy is available for validation.")
except ImportError:
    FINANCEPY_AVAILABLE = False
    print("financepy not installed. Install with: pip install financepy")
    print("Skipping financepy validation.")

In [ ]:
if FINANCEPY_AVAILABLE:
    # Set up financepy pricing
    value_date = Date(1, 1, 2024)
    expiry_date = Date(1, 7, 2024)  # 6 months
    
    discount_curve = DiscountCurveFlat(value_date, r)
    model = BlackScholes(sigma)
    
    call_option = EquityVanillaOption(expiry_date, K, OptionTypes.EUROPEAN_CALL)
    fp_call = call_option.value(value_date, S, discount_curve, q, model)
    
    print("\nfinancepy Validation:")
    print(f"  Our BS Call:      ${our_call:.6f}")
    print(f"  financepy Call:   ${fp_call:.6f}")
    print(f"  Difference:       ${abs(our_call - fp_call):.2e}")
    print(f"  Status: {'✓ MATCH' if abs(our_call - fp_call) < 1e-6 else '✗ MISMATCH'}")

## 3. Tolerance Guidelines [T2]

| Comparison | Tolerance | Reason |
|------------|-----------|--------|
| Same formula (BS vs financepy) | 1e-10 | Floating point only |
| Different conventions (QuantLib) | 1e-8 | Day count differences |
| Different methods (SABR vs pyfeng) | 1e-6 | Numerical differences |
| MC vs analytical | 0.01 | Statistical variance |

## 4. Key Takeaways

1. **Hull textbook** provides golden test cases
2. **financepy** matches to numerical precision
3. **Tolerance varies** by comparison type
4. **Cross-validation builds confidence** in implementation

## References

- Hull, J. C. (2018). Options, Futures, and Other Derivatives, 10th Ed.
- financepy: https://github.com/domokane/FinancePy
- QuantLib: https://www.quantlib.org/